In [1]:
!pip install ipython==7.34.0 ipykernel==5.5.6
!pip install import_ipynb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 6.7 MB/s eta 0:00:00


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import zoom, gaussian_filter
from skimage.filters import threshold_otsu

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, random_split
import math

from google.colab import drive
drive.mount('/content/drive')
import os
import import_ipynb
import imageio

path = '/content/drive/MyDrive/Colab Notebooks/Physics-Informed Neural Networks/Demo/fenics_cfd/neural_network'
os.chdir(path)

Mounted at /content/drive


In [ ]:
from visualize_geometry import plot_numpy_matrices, plot_with_transparent_mask
from data_io import get_joined_files, FlowFieldDataset, train_test_split

In [5]:
class Swish(nn.Module):
    def forward(self, x):
        return x * torch.sigmoid(x)

In [6]:
class SuperResolutionCNN(nn.Module):
    def __init__(self, hidden_layers, dropout_rate=0.5):
        super(SuperResolutionCNN, self).__init__()
        self.swish = Swish()
        self.dropout = nn.Dropout(dropout_rate)

        self.velocity_input_conv = nn.Conv2d(2, hidden_layers[0], kernel_size=3, padding=1)
        self.velocity_input_bn = nn.BatchNorm2d(hidden_layers[0])

        self.pressure_input_conv = nn.Conv2d(1, hidden_layers[0], kernel_size=3, padding=1)
        self.pressure_input_bn = nn.BatchNorm2d(hidden_layers[0])

        self.upsample = nn.Upsample(scale_factor=7, mode='bicubic', align_corners=True)

        self.conv_layers = nn.ModuleList()
        self.bn_layers = nn.ModuleList()

        for i in range(1, len(hidden_layers)):
            self.conv_layers.append(nn.Conv2d(hidden_layers[i-1], hidden_layers[i], kernel_size=3, padding=1))
            self.bn_layers.append(nn.BatchNorm2d(hidden_layers[i]))

        self.velocity_output_conv = nn.Conv2d(hidden_layers[-1], 2, kernel_size=3, padding=1)
        self.pressure_output_conv = nn.Conv2d(hidden_layers[-1], 1, kernel_size=3, padding=1)

    def forward(self, velocity_input, pressure_input):

        velocity = self.velocity_input_conv(velocity_input)
        velocity = self.swish(self.velocity_input_bn(velocity))
        velocity = self.dropout(velocity)
        velocity = self.upsample(velocity)
        for conv_layer, bn_layer in zip(self.conv_layers, self.bn_layers):
            velocity = conv_layer(velocity)
            velocity = self.swish(bn_layer(velocity))
            velocity = self.dropout(velocity)
        velocity_output = self.velocity_output_conv(velocity)

        pressure = self.pressure_input_conv(pressure_input)
        pressure = self.swish(self.pressure_input_bn(pressure))
        pressure = self.dropout(pressure)
        pressure = self.upsample(pressure)
        for conv_layer, bn_layer in zip(self.conv_layers, self.bn_layers):
            pressure = conv_layer(pressure)
            pressure = self.swish(bn_layer(pressure))
            pressure = self.dropout(pressure)
        pressure_output = self.pressure_output_conv(pressure)

        return velocity_output, pressure_output

In [7]:
def relative_error_loss(output, target):
    epsilon = 1e-6
    loss = torch.mean(torch.abs((output - target) / (target + epsilon)))
    return loss


In [8]:
def train_model(model, train_loader, optimizer, criterion, num_epochs, device):
  model.train()

  for epoch in range(num_epochs):
    running_loss = 0.0

    for i, ((u_hr, p_hr), (u_lr, p_lr)) in enumerate(train_loader):

      u_hr, p_hr = u_hr.to(device), p_hr.to(device)
      u_lr, p_lr = u_lr.to(device), p_lr.to(device)

      optimizer.zero_grad()
      u_pred, p_pred = model(u_lr, p_lr)

      loss_velocity = criterion(u_pred, u_hr)
      loss_pressure = criterion(p_pred, p_hr)
      loss = loss_velocity + loss_pressure

      loss.backward()
      optimizer.step()

      # running_loss += loss.item()
      running_loss += loss

      if (i + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_iterations}], Loss: {running_loss / 10}')
        running_loss = 0.0

In [9]:
def test_model(model, test_loader, criterion, num_visualizations, device):
  model.eval()

  with torch.no_grad():

    total_loss = 0
    for i, ((u_hr, p_hr), (u_lr, p_lr)) in enumerate(test_loader):

      u_hr, p_hr = u_hr.to(device), p_hr.to(device)
      u_lr, p_lr = u_lr.to(device), p_lr.to(device)

      u_pred, p_pred = model(u_lr, p_lr)
      loss_velocity = criterion(u_pred, u_hr)
      loss_pressure = criterion(p_pred, p_hr)
      # total_loss += (loss_velocity + loss_pressure).item()
      total_loss += (loss_velocity + loss_pressure)

      if i < num_visualizations:
        u_pred_np = u_pred.cpu().numpy()
        p_pred_np = p_pred.cpu().numpy()
        u_hr_np = u_hr.cpu().numpy()
        p_hr_np = p_hr.cpu().numpy()

        for j in range(u_pred_np.shape[0]):
            plot_numpy_matrices(u_hr_np[j].transpose(1, 2, 0), p_hr_np[j][0], main_title="True Flow Field", plot_size=6)
            plot_numpy_matrices(u_pred_np[j].transpose(1, 2, 0), p_pred_np[j][0], main_title="Predicted Flow Field", plot_size=6)

    print(f'Test Loss: {total_loss / len(test_loader)}')

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f'Device {device}')

if device.type == 'cuda':
  print(torch.cuda.get_device_name())

Device cuda
Tesla T4


In [11]:
npy_velocity_dir = '/content/drive/MyDrive/Colab Notebooks/Physics-Informed Neural Networks/Demo/fenics_cfd/blood_flow_simulations/velocity_fields/npy_data'
npy_pressure_dir = '/content/drive/MyDrive/Colab Notebooks/Physics-Informed Neural Networks/Demo/fenics_cfd/blood_flow_simulations/pressure_fields/npy_data'

joined_files_true = get_joined_files(npy_velocity_dir, npy_pressure_dir, 'true_data')
joined_files_noisy = get_joined_files(npy_velocity_dir, npy_pressure_dir, 'noisy_data')

dataset = FlowFieldDataset(npy_velocity_dir, npy_pressure_dir, joined_files_true, joined_files_noisy)
train_dataset, test_dataset = train_test_split(dataset, test_size=0.2)

In [ ]:
batch_size = 4
num_epochs = 100
total_samples = len(train_dataset)
n_iterations = math.ceil(total_samples / batch_size)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

hidden_layers = np.array([64, 128, 128, 64])
model = SuperResolutionCNN(hidden_layers).to(device)
# criterion = torch.nn.MSELoss()
criterion = relative_error_loss
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)

train_model(model, train_loader, optimizer, criterion, num_epochs, device)

In [17]:
test_model(model, test_loader, criterion, num_visualizations=3, device=device)

Output hidden; open in https://colab.research.google.com to view.